In [1]:
pip install psycopg2

  Using cached psycopg2-2.9.9.tar.gz (384 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [38 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-39
      creating build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/extras.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/extensions.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/_range.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/pool.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/_ipaddress.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/errors.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/__init__.py -> build/lib.linux-x86_64-cpython-39/psycopg2
      copying lib/tz.py -> build/lib.linux-x86_

In [2]:
pip install configparser


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import configparser

In [5]:
config = configparser.ConfigParser()
config.read('db_config.ini')

PGEND_POINT = config['database']['PGEND_POINT']
PGUSER_NAME = config['database']['PGUSER_NAME']
PGPASSWORD = config['database']['PGPASSWORD']
PGDATABASE_NAME = config['database']['PGDATABASE_NAME']

# Setting up the Connection

In [6]:
def connect():

    # Set up a connection to the postgres server.
    conn_string = "host="+ PGEND_POINT +" port="+ "5432" +" dbname="+ PGDATABASE_NAME +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD

    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()

    return conn, cursor

In [7]:
conn, cursor = connect()

Connected!


# Reading the RUL Table

In [8]:
rul_query = sql.SQL("SELECT * FROM rultable;")

# Execute the query
cur = conn.cursor()
cur.execute(rul_query)

# Fetch all rows
rul_data = cur.fetchall()

# Fetch column names from the cursor description
column_names = [desc[0] for desc in cur.description]

# Create a DataFrame with the fetched data and column names
rul_data = pd.DataFrame(rul_data, columns=column_names)

In [9]:
rul_data.head()

,index,remaining_useful_life
0,1,18
1,2,79
2,3,106
3,4,110
4,5,15


In [10]:
rul_data = rul_data.rename(columns={'index': 'engine'})
rul_data = rul_data.rename(columns={'remaining_useful_life': 'RUL'})

In [11]:
rul_data.head()

,engine,RUL
0,1,18
1,2,79
2,3,106
3,4,110
4,5,15


# Reading the Turbo Table

In [12]:
turbo_query = sql.SQL("SELECT * FROM turbotable;")

# Execute the query
cur = conn.cursor()
cur.execute(turbo_query)

# Fetch all rows
turbo_data = cur.fetchall()

# Fetch column names from the cursor description
column_names = [desc[0] for desc in cur.description]

# Create a DataFrame with the fetched data and column names
turbo = pd.DataFrame(turbo_data, columns=column_names)

In [13]:
print(type(turbo))

<class 'pandas.core.frame.DataFrame'>


In [14]:
turbo.columns = turbo.columns.str.lower()

In [15]:
turbo.head()

,index,engine,cycles,altitude (alt),mach_no (mach),throttle_angle (tra),fan_inlet_temp (t2),lpc_outlet_temp (t24),hpc_outlet_temp (t30),lpt_outlet_temp (t50),...,corrected_fan_speed (nrf),corrected_core_speed (nrc),bypass_ratio (bpr),burner_fuel_air_ratio (farb),bleed_enthalpy (htbleed),demanded_fan_speed (nf_dmd),demanded_corrected_fan_speed (pcnfr_dmd),hpt_coolant_bleed (w31),lpt_coolant_bleed (w32),source
0,1,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,...,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071,0
1,5,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,...,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286,0
2,6,1,6,25.0045,0.6205,60.0,462.54,537.02,1266.38,1048.72,...,2028.01,7868.87,10.8912,0.02,306,1915,84.93,14.28,8.5590,0
3,8,1,8,20.0020,0.7002,100.0,491.19,607.44,1481.69,1252.36,...,2387.99,8049.26,9.2369,0.02,365,2324,100.00,24.33,14.7989,0
4,13,1,13,20.0003,0.7000,100.0,491.19,607.67,1488.74,1256.10,...,2388.04,8048.52,9.1968,0.02,365,2324,100.00,24.40,14.7202,0


In [16]:
turbo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87750 entries, 0 to 87749
Data columns (total 28 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   index                                     87750 non-null  int64  
 1   engine                                    87750 non-null  int64  
 2   cycles                                    87750 non-null  int64  
 3   altitude (alt)                            87750 non-null  float64
 4   mach_no (mach)                            87750 non-null  float64
 5   throttle_angle (tra)                      87750 non-null  float64
 6   fan_inlet_temp (t2)                       87750 non-null  float64
 7   lpc_outlet_temp (t24)                     87750 non-null  float64
 8   hpc_outlet_temp (t30)                     87750 non-null  float64
 9   lpt_outlet_temp (t50)                     87750 non-null  float64
 10  fan_inlet_pressure (p2)           

In [17]:
turbo.describe().T

,count,mean,std,min,25%,50%,75%,max
index,87750.0,43875.500000,25331.387398,1.0000,21938.25000,43875.50000,65812.7500,87750.0000
engine,87750.0,129.706051,74.360146,1.0000,66.00000,129.00000,192.0000,260.0000
cycles,87750.0,98.335305,66.788208,1.0000,43.00000,90.00000,143.0000,378.0000
altitude (alt),87750.0,23.989409,14.759538,0.0000,10.00460,25.00130,41.9980,42.0080
mach_no (mach),87750.0,0.571707,0.310395,0.0000,0.25060,0.70000,0.8400,0.8420
throttle_angle (tra),87750.0,94.054017,14.229790,60.0000,100.00000,100.00000,100.0000,100.0000
fan_inlet_temp (t2),87750.0,472.917893,26.414747,445.0000,445.00000,462.54000,491.1900,518.6700
lpc_outlet_temp (t24),87750.0,579.641141,37.308401,535.4100,549.54000,555.92000,607.3100,644.5200
hpc_outlet_temp (t30),87750.0,1419.425250,105.960678,1243.7300,1352.31000,1368.45000,1498.9000,1612.8800
lpt_outlet_temp (t50),87750.0,1204.529599,119.095741,1023.7700,1123.17000,1137.14500,1306.2500,1439.2300


In [18]:
turbo.shape

(87750, 28)

In [19]:
# Split the dataset based on the value of the source column
train_data = turbo[turbo['source'] == 0]
test_data = turbo[turbo['source'] == 1]

In [20]:
print(type(train_data))
print(type(test_data))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [21]:
train_data.head()

,index,engine,cycles,altitude (alt),mach_no (mach),throttle_angle (tra),fan_inlet_temp (t2),lpc_outlet_temp (t24),hpc_outlet_temp (t30),lpt_outlet_temp (t50),...,corrected_fan_speed (nrf),corrected_core_speed (nrc),bypass_ratio (bpr),burner_fuel_air_ratio (farb),bleed_enthalpy (htbleed),demanded_fan_speed (nf_dmd),demanded_corrected_fan_speed (pcnfr_dmd),hpt_coolant_bleed (w31),lpt_coolant_bleed (w32),source
0,1,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,...,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071,0
1,5,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,...,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286,0
2,6,1,6,25.0045,0.6205,60.0,462.54,537.02,1266.38,1048.72,...,2028.01,7868.87,10.8912,0.02,306,1915,84.93,14.28,8.5590,0
3,8,1,8,20.0020,0.7002,100.0,491.19,607.44,1481.69,1252.36,...,2387.99,8049.26,9.2369,0.02,365,2324,100.00,24.33,14.7989,0
4,13,1,13,20.0003,0.7000,100.0,491.19,607.67,1488.74,1256.10,...,2388.04,8048.52,9.1968,0.02,365,2324,100.00,24.40,14.7202,0


In [22]:
train_data.columns

Index(['index', 'engine', 'cycles', 'altitude (alt)', 'mach_no (mach)',
       'throttle_angle (tra)', 'fan_inlet_temp (t2)', 'lpc_outlet_temp (t24)',
       'hpc_outlet_temp (t30)', 'lpt_outlet_temp (t50)',
       'fan_inlet_pressure (p2)', 'bypass_duct_pressure (p15)',
       'hpc_outlet_pressure (p30)', 'fan_speed (nf)', 'core_speed (nc)',
       'engine_pressure_ratio (epr)', 'hpc_outlet_static_pressure (ps30)',
       'fuel_ps30_ratio (phi)', 'corrected_fan_speed (nrf)',
       'corrected_core_speed (nrc)', 'bypass_ratio (bpr)',
       'burner_fuel_air_ratio (farb)', 'bleed_enthalpy (htbleed)',
       'demanded_fan_speed (nf_dmd)',
       'demanded_corrected_fan_speed (pcnfr_dmd)', 'hpt_coolant_bleed (w31)',
       'lpt_coolant_bleed (w32)', 'source'],
      dtype='object')

In [23]:
train_data.shape

(53759, 28)

In [24]:
test_data.head()

,index,engine,cycles,altitude (alt),mach_no (mach),throttle_angle (tra),fan_inlet_temp (t2),lpc_outlet_temp (t24),hpc_outlet_temp (t30),lpt_outlet_temp (t50),...,corrected_fan_speed (nrf),corrected_core_speed (nrc),bypass_ratio (bpr),burner_fuel_air_ratio (farb),bleed_enthalpy (htbleed),demanded_fan_speed (nf_dmd),demanded_corrected_fan_speed (pcnfr_dmd),hpt_coolant_bleed (w31),lpt_coolant_bleed (w32),source
53758,53763,1,4,42.0066,0.8410,100.0,445.00,550.17,1349.69,1127.89,...,2387.72,8066.90,9.3961,0.02,332,2212,100.0,10.35,6.4181,1
53760,53760,1,1,9.9987,0.2502,100.0,489.05,605.03,1497.17,1304.99,...,2388.18,8114.10,8.6476,0.03,369,2319,100.0,28.42,17.1551,1
53761,53761,1,2,20.0026,0.7000,100.0,491.19,607.82,1481.20,1246.11,...,2388.12,8053.06,9.2405,0.02,364,2324,100.0,24.29,14.8039,1
53762,53762,1,3,35.0045,0.8400,100.0,449.44,556.00,1359.08,1128.36,...,2387.75,8053.04,9.3472,0.02,333,2223,100.0,14.98,8.9125,1
53763,53769,1,10,20.0001,0.7000,100.0,491.19,607.33,1483.66,1261.66,...,2388.03,8045.67,9.2522,0.02,365,2324,100.0,24.47,14.6186,1


In [25]:
test_data.shape

(33991, 28)

In [26]:
test_data['engine'].nunique()

259

In [27]:
# Calculate RUL for training data
train_data['RUL'] = train_data.groupby('engine')['cycles'].transform(max) - train_data['cycles']

In [28]:
train_data.columns

Index(['index', 'engine', 'cycles', 'altitude (alt)', 'mach_no (mach)',
       'throttle_angle (tra)', 'fan_inlet_temp (t2)', 'lpc_outlet_temp (t24)',
       'hpc_outlet_temp (t30)', 'lpt_outlet_temp (t50)',
       'fan_inlet_pressure (p2)', 'bypass_duct_pressure (p15)',
       'hpc_outlet_pressure (p30)', 'fan_speed (nf)', 'core_speed (nc)',
       'engine_pressure_ratio (epr)', 'hpc_outlet_static_pressure (ps30)',
       'fuel_ps30_ratio (phi)', 'corrected_fan_speed (nrf)',
       'corrected_core_speed (nrc)', 'bypass_ratio (bpr)',
       'burner_fuel_air_ratio (farb)', 'bleed_enthalpy (htbleed)',
       'demanded_fan_speed (nf_dmd)',
       'demanded_corrected_fan_speed (pcnfr_dmd)', 'hpt_coolant_bleed (w31)',
       'lpt_coolant_bleed (w32)', 'source', 'RUL'],
      dtype='object')

In [29]:
df = test_data.merge(rul_data[['engine', 'RUL']], on='engine', how='left')

In [30]:
df.columns

Index(['index', 'engine', 'cycles', 'altitude (alt)', 'mach_no (mach)',
       'throttle_angle (tra)', 'fan_inlet_temp (t2)', 'lpc_outlet_temp (t24)',
       'hpc_outlet_temp (t30)', 'lpt_outlet_temp (t50)',
       'fan_inlet_pressure (p2)', 'bypass_duct_pressure (p15)',
       'hpc_outlet_pressure (p30)', 'fan_speed (nf)', 'core_speed (nc)',
       'engine_pressure_ratio (epr)', 'hpc_outlet_static_pressure (ps30)',
       'fuel_ps30_ratio (phi)', 'corrected_fan_speed (nrf)',
       'corrected_core_speed (nrc)', 'bypass_ratio (bpr)',
       'burner_fuel_air_ratio (farb)', 'bleed_enthalpy (htbleed)',
       'demanded_fan_speed (nf_dmd)',
       'demanded_corrected_fan_speed (pcnfr_dmd)', 'hpt_coolant_bleed (w31)',
       'lpt_coolant_bleed (w32)', 'source', 'RUL'],
      dtype='object')

In [31]:
df.shape

(33991, 29)

In [32]:
# Function to clean column names
def clean_column_name(name):
    return name.split(' ')[0]

# Apply the function to rename columns
df.columns = [clean_column_name(col) for col in df.columns]

df.rename(columns={'RUL': 'rul'}, inplace=True)


print(df.columns)

Index(['index', 'engine', 'cycles', 'altitude', 'mach_no', 'throttle_angle',
       'fan_inlet_temp', 'lpc_outlet_temp', 'hpc_outlet_temp',
       'lpt_outlet_temp', 'fan_inlet_pressure', 'bypass_duct_pressure',
       'hpc_outlet_pressure', 'fan_speed', 'core_speed',
       'engine_pressure_ratio', 'hpc_outlet_static_pressure',
       'fuel_ps30_ratio', 'corrected_fan_speed', 'corrected_core_speed',
       'bypass_ratio', 'burner_fuel_air_ratio', 'bleed_enthalpy',
       'demanded_fan_speed', 'demanded_corrected_fan_speed',
       'hpt_coolant_bleed', 'lpt_coolant_bleed', 'source', 'rul'],
      dtype='object')


In [33]:
train_data.columns = [clean_column_name(col) for col in train_data.columns]

train_data.rename(columns={'RUL': 'rul'}, inplace=True)

print(train_data.columns)

Index(['index', 'engine', 'cycles', 'altitude', 'mach_no', 'throttle_angle',
       'fan_inlet_temp', 'lpc_outlet_temp', 'hpc_outlet_temp',
       'lpt_outlet_temp', 'fan_inlet_pressure', 'bypass_duct_pressure',
       'hpc_outlet_pressure', 'fan_speed', 'core_speed',
       'engine_pressure_ratio', 'hpc_outlet_static_pressure',
       'fuel_ps30_ratio', 'corrected_fan_speed', 'corrected_core_speed',
       'bypass_ratio', 'burner_fuel_air_ratio', 'bleed_enthalpy',
       'demanded_fan_speed', 'demanded_corrected_fan_speed',
       'hpt_coolant_bleed', 'lpt_coolant_bleed', 'source', 'rul'],
      dtype='object')


In [34]:
# Extract the last cycle for each engine in the test set
test_last_cycles = df.groupby('engine')['cycles'].idxmax()
test_last_cycles_df = df.loc[test_last_cycles]

In [35]:
test_last_cycles_df.columns

Index(['index', 'engine', 'cycles', 'altitude', 'mach_no', 'throttle_angle',
       'fan_inlet_temp', 'lpc_outlet_temp', 'hpc_outlet_temp',
       'lpt_outlet_temp', 'fan_inlet_pressure', 'bypass_duct_pressure',
       'hpc_outlet_pressure', 'fan_speed', 'core_speed',
       'engine_pressure_ratio', 'hpc_outlet_static_pressure',
       'fuel_ps30_ratio', 'corrected_fan_speed', 'corrected_core_speed',
       'bypass_ratio', 'burner_fuel_air_ratio', 'bleed_enthalpy',
       'demanded_fan_speed', 'demanded_corrected_fan_speed',
       'hpt_coolant_bleed', 'lpt_coolant_bleed', 'source', 'rul'],
      dtype='object')

In [36]:
test_last_cycles_df.shape

(259, 29)

In [37]:
train_data.shape

(53759, 29)

In [38]:
import psycopg2
from sqlalchemy import create_engine
 
# Merging the train and test DataFrames
combined_df = pd.concat([train_data, test_last_cycles_df], ignore_index=True)
 
# Write the combined DataFrame to RDS
def write_to_rds(df, table_name):
    config = configparser.ConfigParser()
    config.read('db_config.ini')
 
    PGEND_POINT = config['database']['PGEND_POINT']
    PGUSER_NAME = config['database']['PGUSER_NAME']
    PGPASSWORD = config['database']['PGPASSWORD']
    PGDATABASE_NAME = config['database']['PGDATABASE_NAME']
 
    # Create SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{PGUSER_NAME}:{PGPASSWORD}@{PGEND_POINT}:5432/{PGDATABASE_NAME}')
 
    # Write the DataFrame to PostgreSQL
    df.to_sql(table_name, engine, if_exists='replace', index=False)
 
# Assuming the table name is 'combined_turbofan_data'
write_to_rds(combined_df, 'turbofan_engine_with_rul')